In [2]:
import numpy as np  
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import os
from astropy.wcs import WCS
from photutils.aperture import CircularAperture, CircularAnnulus
from photutils.aperture import aperture_photometry

### Factor F

In [3]:
GAIAspectrum = fits.open('GAIA3_509862066010920960.fits')

def Factor(filtro):

    file_name = f"{filtro}_trasmission.txt"

    r_trasm = Table.read(file_name, format='ascii.basic')
    r_wave=r_trasm["Wavelength(A)"]
    r_value=r_trasm["Transmission(QE*Filter)"]

    r_trasm_new=np.interp(GAIAspectrum[1].data, r_wave, r_value)

    x=GAIAspectrum[1].data
    y_filter=r_trasm_new
    y_spectrum=np.array(GAIAspectrum[0].data)

    Num = np.trapz(y_spectrum * y_filter, x)
    Den= np.trapz(y_filter, x)
    return Num/Den           # Factor for the flux

### Bias 
Independent on anything 


In [ ]:
path=os.getcwd()+'/bias'
fitsfile=fits.open(path+'/'+'16-23-54_Ha_Bias_0.00s_0000.fits')         #assuming all bias have the same shape
datax = fitsfile[0].data

def bias():
    path=os.getcwd()+'/bias'
    Num_of_files=len(os.listdir(path))
    Grid=np.zeros(datax.shape)
    for file in os.listdir(path):
        fitsfile=fits.open(path+'/'+file)
        dati = fitsfile[0].data
        Grid+=dati
    Grid/=Num_of_files
    return Grid

# it is the same for all the images 
masterbias=bias()

### Flat

In [5]:
def flat(filtro):   

    pathf=os.getcwd()+f"/{filtro}FLAT"
    fitsfilef = fits.open(pathf + f"/2025-10-02_09-25-17_FLAT_{filtro}__0000.fits")       #assuming all flat have the same shape
    dataf = fitsfilef[0].data

    Num_of_files=len(os.listdir(pathf))
    Grid=np.zeros((dataf.shape[0],dataf.shape[1],Num_of_files))
    j=0
    for file in os.listdir(pathf):
        fitsfile=fits.open(pathf+'/'+file)
        dati = fitsfile[0].data
        Grid[:,:,j]=dati
        j+=1
    master=np.median(Grid, axis=2)-masterbias
    
    return master/np.median(master)

masterflat=flat('R')

### Dark current

In [6]:
def dark_current(exptime):

    pathd=os.getcwd()+'/DARK'+f'{exptime}'
    fitsfiled=fits.open(pathd+'/'+'2025-09-25_17-59-33_DARK_300.00s_0000.fits')
    datad = fitsfiled[0].data

    Num_of_files=len(os.listdir(pathd))
    Grid=np.zeros((datad.shape[0],datad.shape[1],Num_of_files))
    j=0
    for file in os.listdir(pathd):
        fitsfile=fits.open(pathd+'/'+file)
        dati = fitsfile[0].data
        Grid[:,:,j]=dati
        j+=1
    master=np.median(Grid, axis=2)-masterbias
    return master

mastercurrent=dark_current('300')

In [ ]:
pixpos = wcs.wcs_world2pix(23.38804096055661,60.63702187961969, 1)

data_cut = data2[1700:1800,2400:2500]
        position = pixpos[0]-2400, pixpos[1]-1700  # (x, y) position of the star in the cutout
        Apertures = [CircularAperture(position, r=i) for i in np.arange(1, 30, 0.5)]
fluxs=aperture_photometry(sci_cut,Apertures)

In [ ]:
def DataProcess(filter, exptime):

    Path=os.getcwd()+'/DATA'+'/f{filter}'

    masterbias=bias()
    mastercurrent=dark_current('exptime')
    masterflat=flat('filter')

    for file in os.listdir(Path):       #for all the file in the filter of a given day

        Fitsfile=fits.open(Path+'/'+file)

        #read the data 
        Data = Fitsfile[0].data

        #coordinates of the image
        wcs = WCS(Fitsfile[0].header)

        data1 = (Data-masterbias-mastercurrent)/masterflat
        data2=data1*0.25/300                                    # to have the flux in e-/s     
        
        



In [16]:
DataProcess('R',300)

/Users/sofiadossena/Desktop/LaboratoryOfDataAcquisition/DATA/f{filter}
